In [1]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV

In [ ]:
#load cost data
con_generation = CSV.File("../data/conventional_generators.csv") |> DataFrame

#load wind_technicaldata
wind_generation = CSV.File("../data/wind_farms.csv") |> DataFrame

# load wind profile
wind_profile = CSV.File("../data/wind_powerprofile_200.csv") |> DataFrame

# batteries
battery = CSV.File("../data/Battery.csv") |> DataFrame


In [ ]:
#demand bids
name = "demand_hour_0.csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
 
# number of convential generators
G = size(con_generation, 1)

# number of demand
D = size(demand_bids, 1)

# number of wind generators
W = size(wind_generation, 1)

# number of batteries
B = size(battery, 1)


In [ ]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[])
equilibrium_df = DataFrame(hour = Int[], market_price = Float64[], total_demand = Float64[], total_bat = Float64[], total_supply = Float64[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    result_df[!, Symbol("x_con$i")] = Float64[]
end

# For w variables, manually add each column. This is a one-time setup.
for i in 1:W
    result_df[!, Symbol("x_wind$i")] = Float64[]
end

# For y variables, manually add each column. This is a one-time setup.
for i in 1:D
    result_df[!, Symbol("y$i")] = Float64[]
end

# For b variables, manually add each column. This is a one-time setup.
for i in 1:B
    result_df[!, Symbol("x_bat_charg$i")] = Float64[]
end

# For b variables, manually add each column. This is a one-time setup.
for i in 1:B
    result_df[!, Symbol("x_bat_disch$i")] = Float64[]
end


In [ ]:
println(con_generation)

In [ ]:
# Load demand bids data
demand_bids_all = [CSV.File("../data/demand_bids_hour/demand_hour_$(i-1).csv") |> DataFrame for i in 1:24]

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for every generator and hour
@variable(model, x_con[1:G, 1:24] >= 0)  # Power output variable for conventional generators
@variable(model, x_wind[1:W, 1:24] >= 0)  # Power output variable for wind generators
@variable(model, y[1:D, 1:24] >= 0)  # Demand variable
@variable(model, bat_char[1:B, 1:24])  # Battery variable when it is charging
@variable(model, bat_disch[1:B, 1:24])  # Battery variable when it is discharging

# Add initial values for the battery
init_bat = []
for b in 1:B
    append!(init_bat, 0)
end

# Define the charging and discharging efficiencies of the battery
eff_char = 0.9
eff_disch= 0.9



# Add constraints for each plant
for g in 1:G
    for hour in 1:24
        @constraint(model, x_con[g, hour] <= con_generation[g, 6])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each wind farm
for w in 1:W
    for hour in 1:24
        @constraint(model, x_wind[w, hour] <= wind_profile[hour, w+1])  # Set the upper bound
        @constraint(model, x_wind[w, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each demand bid
for d in 1:D
    for hour in 1:24
        @constraint(model, y[d, hour] <= demand_bids_all[hour][d, 2])  # Set the upper bound
        @constraint(model, y[d, hour] >= 0)  # Set the lower bound
    end
end

#add constraints for initial con_generation
for g in 1:G
    @constraint(model, x_con[g, 1] <= con_generation[g, 11] + con_generation[g, 9])  # Set the upper bound
    @constraint(model, x_con[g, 1] >= con_generation[g, 11] - con_generation[g, 10])  # Set the lower bound
end

# Add temporary for each generator
for g in 1:G
    for hour in 2:24
        @constraint(model, x_con[g, hour] <= x_con[g, hour - 1] + con_generation[g, 9])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= x_con[g, hour - 1] - con_generation[g, 10])  # Set the lower bound
    end
end

# add Power constraints for battery charging
for b in 1:B
    for hour in 1:24
        @constraint(model, bat_char[b, hour] <= battery[b, 5])  # Set the upper bound
        @constraint(model, bat_char[b, hour] >= 0)  # Set the lower bound 
    end
end

# add Power constraints for battery discharging, with positive values
for b in 1:B
    for hour in 1:24
        @constraint(model, bat_disch[b, hour] <= battery[b, 5])  # Set the upper bound
        @constraint(model, bat_disch[b, hour] >= 0)  # Set the lower bound
    end
end

# add Capacity constraints for battery charging and discharging
# sum all previous battery charging and discharging, they need to be below the battery capacity
for b in 1:B
    for hour in 1:24
        @constraint(model, sum(init_bat[b] + bat_char[b, h]*eff_char - bat_disch[b, h]/eff_disch for h in 1:hour) <= battery[b, 4])  # Set the upper bound     ##Add initial value + for 1 to 24 instead of 2 to 24
        @constraint(model, sum(init_bat[b] + bat_char[b, h]*eff_char - bat_disch[b, h]/eff_disch for h in 1:hour) >= 0)  # Set the lower bound
    end
end

# Add the constraint for the balance between supply and demand for each hour
@constraint(model, balance[hour = 1:24], sum(x_con[g, hour] for g in 1:G)
    + sum(x_wind[w, hour] for w in 1:W) - sum(y[d, hour] for d in 1:D)
    - sum(bat_char[b, hour] for b in 1:B)
    + sum(bat_disch[b, hour] for b in 1:B) == 0)  

# Define the objective function
@objective(model, Max, sum(demand_bids_all[hour][d, 3] * y[d, hour] for hour in 1:24, d in 1:D) -
                     sum(con_generation[g, 3] * x_con[g, hour] for hour in 1:24, g in 1:G) - 
                     sum(0.0001 * bat_char[b, hour] for hour in 1:24, b in 1:B) -
                     sum(0.0001 * bat_disch[b, hour] for hour in 1:24, b in 1:B) ) # Add the penalty of battery charging and discharging


# Solve the model
optimize!(model)

# Clear the DataFrame equilibrium_df   ## Clear the dataframe
empty!(equilibrium_df)

# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    #save the results in result and equilibrium dataframe
    for hour in 1:24
        push!(equilibrium_df, [hour, dual(balance[hour]), sum(value(y[i, hour]) for i in 1:D), sum(init_bat[b] + value(bat_char[b, h])*eff_char - value(bat_disch[b, h])/eff_disch for b in 1:B, h in 1:hour), sum(value(x_con[i, hour]) for i in 1:G) + sum(value(x_wind[i, hour]) for i in 1:W)])
    end

    #save results to dataframe
    for hour in 1:24
        push!(result_df, [hour, [value(x_con[i, hour]) for i in 1:G]..., [value(x_wind[i, hour]) for i in 1:W]..., [value(y[i, hour]) for i in 1:D]..., [value(bat_char[i, hour]) for i in 1:B]..., [value(bat_disch[i, hour]) for i in 1:B]...])
    end

    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))


else
    println("No optimal solution found")
end


In [ ]:
# make a df for bat_result
bat_result_df = DataFrame(hour = Int[])

# For b variables, manually add each column. This is a one-time setup.
for i in 1:B
    bat_result_df[!, Symbol("x_bat_char$i")] = Float64[]
    bat_result_df[!, Symbol("x_bat_disch$i")] = Float64[]
end


#save results to dataframe alternating between charging and discharging
for hour in 1:24
    resultvector = []
    for i in 1:B
        append!(resultvector, value(bat_char[i, hour]))
        append!(resultvector, value(bat_disch[i, hour]))
    end
    push!(bat_result_df, [hour, resultvector...])
end

In [ ]:
#Save bat_result to csv
CSV.write("results/bat_result.csv", bat_result_df)

In [ ]:
#sum columns of bat_result_df
bat_result_df

In [ ]:
equilibrium_df

In [ ]:
#save equilibrium_df to csv
CSV.write("results/equilibrium_model2.csv", equilibrium_df)

In [ ]:
# fill the values for the result_df
for i in 1:G
    result_df[!, Symbol("x_con$i")] = [value(x_con[i, hour]) for hour in 1:24]
end

for i in 1:W
    result_df[!, Symbol("x_wind$i")] = [value(x_wind[i, hour]) for hour in 1:24]
end

for i in 1:D
    result_df[!, Symbol("y$i")] = [value(y[i, hour]) for hour in 1:24]
end

for i in 1:B
    result_df[!, Symbol("x_bat_charg$i")] = [value(bat_char[i, hour]) for hour in 1:24]
end

for i in 1:B
    result_df[!, Symbol("x_bat_disch$i")] = [value(bat_disch[i, hour]) for hour in 1:24]
end

result_df



In [ ]:
#plot equilibrium_df market_price in a line plot
using Plots
plot(equilibrium_df.hour, equilibrium_df.market_price, label="Market Price", xlabel="Hour", ylabel="Price", title="Market Price by Hour", lw=2)

In [ ]:
#plot equilibrium_df total_demand and total_supply in a line plot
plot(equilibrium_df.hour, [equilibrium_df.total_demand equilibrium_df.total_supply], label=["Total Demand" "Total Supply"], xlabel="Hour", ylabel="Power", lw=2)

#save the plot
savefig("results/supply_demand_model2.png")


In [ ]:
#Plot equilibrium_df total_bat in a line plot
plot(equilibrium_df.hour, equilibrium_df.total_bat, label="Total Battery", xlabel="Hour", ylabel="Capacity of the batteries", title="Total Battery by Hour", lw=2)

In [ ]:
#save result_df to csv
CSV.write("results/market_clearing_model2.csv", result_df)